In [3]:
import numpy as np
import pandas as pd
from sklearn.model_selection import train_test_split
from keras.preprocessing.text import Tokenizer
# from keras.preprocessing.sequence import pad_sequences
from tensorflow.keras.preprocessing.sequence import pad_sequences
from keras.layers import Embedding, LSTM, Dense
from keras.models import Sequential

In [8]:
# Load dataset
data = pd.read_csv('complaintsdata.csv')

In [17]:
data['l2'].unique()


array(['apparel', 'electronics', 'other', 'food', 'software', 'cars',
       'retail', 'services', 'random_tweet', 'random_reply', 'transport'],
      dtype=object)

In [10]:
# Preprocessing
tokenizer = Tokenizer(num_words=5000)
tokenizer.fit_on_texts(data['text'])
X = tokenizer.texts_to_sequences(data['text'])
X = pad_sequences(X, maxlen=100)

In [11]:
# Train-Test Split
y = pd.get_dummies(data['l2']).values
X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.2, random_state=42)

In [12]:
# Word Embedding
embedding_matrix = np.random.rand(5000, 100)
embedding_layer = Embedding(5000, 100, weights=[embedding_matrix], input_length=100, trainable=False)


In [18]:
# Model Building
model = Sequential()
model.add(embedding_layer)
model.add(LSTM(100, dropout=0.2, recurrent_dropout=0.2))
model.add(Dense(11, activation='softmax'))
model.compile(loss='categorical_crossentropy', optimizer='adam', metrics=['accuracy'])
print(model.summary())

Model: "sequential_1"
_________________________________________________________________
 Layer (type)                Output Shape              Param #   
 embedding (Embedding)       (None, 100, 100)          500000    
                                                                 
 lstm_1 (LSTM)               (None, 100)               80400     
                                                                 
 dense_1 (Dense)             (None, 11)                1111      
                                                                 
Total params: 581,511
Trainable params: 81,511
Non-trainable params: 500,000
_________________________________________________________________
None


In [19]:
# Model Training
model.fit(X_train, y_train, epochs=10, batch_size=32, validation_data=(X_test, y_test))


Epoch 1/10
87/87 [==============================] - 9s 75ms/step - loss: 2.1340 - accuracy: 0.2751 - val_loss: 2.0651 - val_accuracy: 0.3261
Epoch 2/10
87/87 [==============================] - 6s 68ms/step - loss: 2.0207 - accuracy: 0.3135 - val_loss: 2.0253 - val_accuracy: 0.3348
Epoch 3/10
87/87 [==============================] - 6s 70ms/step - loss: 1.9881 - accuracy: 0.3161 - val_loss: 2.0128 - val_accuracy: 0.3174
Epoch 4/10
87/87 [==============================] - 6s 71ms/step - loss: 1.9771 - accuracy: 0.3222 - val_loss: 1.9982 - val_accuracy: 0.3420
Epoch 5/10
87/87 [==============================] - 6s 71ms/step - loss: 1.9565 - accuracy: 0.3273 - val_loss: 2.0043 - val_accuracy: 0.3333
Epoch 6/10
87/87 [==============================] - 6s 70ms/step - loss: 1.9573 - accuracy: 0.3313 - val_loss: 2.0366 - val_accuracy: 0.3319
Epoch 7/10
87/87 [==============================] - 6s 71ms/step - loss: 1.9300 - accuracy: 0.3396 - val_loss: 1.9717 - val_accuracy: 0.3507
Epoch 8/10
87

In [20]:
# Model Evaluation
scores = model.evaluate(X_test, y_test, verbose=0)
print("Accuracy: %.2f%%" % (scores[1]*100))

Accuracy: 33.91%
